In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [ ]:
import os
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [ ]:
import os
import zipfile
import random
import shutil
from sklearn.model_selection import train_test_split

zip_path = '/content/drive/MyDrive/paper/archive (3).zip'
extract_path = '/content/drive/MyDrive/data'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

base_dir = '/content/retinopathy_split'
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'val')
test_dir = os.path.join(base_dir, 'test')

for directory in [train_dir, val_dir, test_dir]:
    if not os.path.exists(directory):
        os.makedirs(directory)

classes = ['0', '1', '2', '3', '4']

for c in classes:
    for split in [train_dir, val_dir, test_dir]:
        os.makedirs(os.path.join(split, c), exist_ok=True)

for c in classes:
    class_dir = os.path.join(extract_path, c)
    images = os.listdir(class_dir)
    random.shuffle(images)

    train_split = int(0.7 * len(images))
    val_split = int(0.85 * len(images))

    train_images = images[:train_split]
    val_images = images[train_split:val_split]
    test_images = images[val_split:]

    # Image translation
    for img in train_images:
        shutil.copy(os.path.join(class_dir, img), os.path.join(train_dir, c, img))
    for img in val_images:
        shutil.copy(os.path.join(class_dir, img), os.path.join(val_dir, c, img))
    for img in test_images:
        shutil.copy(os.path.join(class_dir, img), os.path.join(test_dir, c, img))

print("The dataset was successfully divided✅")

The dataset was successfully divided✅


In [ ]:
import torch
from torchvision import transforms, datasets
from torch.utils.data import DataLoader

# ⚡️ Training-specific augmentations
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# 🌱 Transformations for validation and testing (without augmentations)
eval_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# 📂 Data loading
train_data = datasets.ImageFolder(root='/content/retinopathy_split/train', transform=train_transform)
val_data = datasets.ImageFolder(root='/content/retinopathy_split/val', transform=eval_transform)
test_data = datasets.ImageFolder(root='/content/retinopathy_split/test', transform=eval_transform)

# 🧺 Data Loaders
train_loader = DataLoader(train_data, batch_size=32, shuffle=True, num_workers=2)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False, num_workers=2)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False, num_workers=2)


In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models

# Loading the MobileNetV2 model pretrained on ImageNet
mobilenet = models.mobilenet_v2(pretrained=True)

# Adding a higher Dropout rate
mobilenet.classifier[0] = nn.Dropout(p=0.5)
mobilenet.classifier[1] = nn.Linear(mobilenet.classifier[1].in_features, 5)

# Transfer the model to the GPU if it is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
mobilenet = mobilenet.to(device)

# Loss Function and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(mobilenet.parameters(), lr=1e-4)

# Learning rate decay during training
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.5)



/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth
100%|██████████| 13.6M/13.6M [00:00<00:00, 125MB/s]


In [ ]:
def evaluate(model, loader):
    model.eval()
    total_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    avg_loss = total_loss / len(loader)
    return avg_loss, accuracy

num_epochs = 20

for epoch in range(num_epochs):
    mobilenet.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = mobilenet(images)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_loss = running_loss / len(train_loader)
    train_acc = 100 * correct / total

    val_loss, val_acc = evaluate(mobilenet, val_loader)
    test_loss, test_acc = evaluate(mobilenet, test_loader)

    print(f"Epoch [{epoch+1}/{num_epochs}]")
    print(f"  Train     => Loss: {train_loss:.4f}, Accuracy: {train_acc:.2f}%")
    print(f"  Validation=> Loss: {val_loss:.4f}, Accuracy: {val_acc:.2f}%")
    print(f"  Test      => Loss: {test_loss:.4f}, Accuracy: {test_acc:.2f}%")
    print("-" * 60)


Epoch [1/20]
  Train     => Loss: 0.7976, Accuracy: 73.41%
  Validation=> Loss: 0.6947, Accuracy: 76.14%
  Test      => Loss: 0.6965, Accuracy: 76.74%
------------------------------------------------------------


In [ ]:
from sklearn.model_selection import ParameterGrid

param_grid = {
    'batch_size': [16, 32],
    'lr': [1e-3, 1e-4],
    'dropout': [0.3, 0.5]
}

grid = list(ParameterGrid(param_grid))


In [ ]:
!pip install pytorch-gradcam

In [ ]:
pip install captum

In [ ]:
!pip install torch torchvision captum matplotlib numpy

import torch
from torchvision import datasets, transforms
from captum.attr import IntegratedGradients
import matplotlib.pyplot as plt
import numpy as np
import random

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = torch.hub.load('pytorch/vision:v0.10.0', 'mobilenet_v2', pretrained=True)
model.classifier[1] = torch.nn.Linear(model.classifier[1].in_features, 5)
model = model.to(device)
model.eval()

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

test_dataset = datasets.ImageFolder(
    '/content/retinopathy_split/test',
    transform=transform
)

def show_image(img_tensor):
    img = img_tensor.cpu().numpy().transpose(1, 2, 0)
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    img = std * img + mean
    img = np.clip(img, 0, 1)
    plt.imshow(img)
    plt.axis('off')
    plt.show()

def visualize_attr(attr):
    attr = attr.cpu().detach().numpy()

    if attr.ndim == 4:
        attr = attr.squeeze(0)
    if attr.ndim == 3 and attr.shape[0] == 1:
        attr = attr.squeeze(0)
    elif attr.ndim == 3 and attr.shape[0] == 3:
        attr = attr.transpose(1, 2, 0)
        attr = np.sum(np.abs(attr), axis=2)
    elif attr.ndim == 2:
        pass
    else:
        raise ValueError(f"شكل غير متوقع للبيانات: {attr.shape}")

    attr = (attr - attr.min()) / (attr.max() - attr.min() + 1e-8)

    plt.imshow(attr, cmap='hot')
    plt.colorbar()
    plt.axis('off')
    plt.title('Importance Map')
    plt.show()

idx = random.randint(0, len(test_dataset)-1)
image, label = test_dataset[idx]
image = image.to(device)
label = torch.tensor(label).to(device)

ig = IntegratedGradients(model)
attributions, delta = ig.attribute(
    inputs=image.unsqueeze(0),
    target=label,
    return_convergence_delta=True
)

print(f"Convergence Delta: {delta.item():.4f}")
show_image(image)
visualize_attr(attributions)


In [ ]:
# ثانياً: استيراد المكتبات
import torch
from torchvision import datasets, transforms
from captum.attr import IntegratedGradients
import matplotlib.pyplot as plt
import numpy as np
import random

# ثالثاً: تعريف الجهاز (GPU إذا متوفر)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# رابعاً: تحميل النموذج المدرب (يجب استبدال هذا الجزء بنموذجك الخاص)
model = torch.hub.load('pytorch/vision:v0.10.0', 'mobilenet_v2', pretrained=True)
model.classifier[1] = torch.nn.Linear(model.classifier[1].in_features, 5)  # تعديل ل 5 فئات
model = model.to(device)
model.eval()

# خامساً: إعداد تحويلات الصور
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# سادساً: تحميل بيانات الاختبار
test_dataset = datasets.ImageFolder(
    '/content/retinopathy_split/test',  # تعديل المسار حسب بياناتك
    transform=transform
)

# سابعاً: دالة لعرض الصورة الأصلية
def show_image(img_tensor):
    img = img_tensor.cpu().numpy().transpose(1, 2, 0)
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    img = std * img + mean  # عكس التطبيع
    img = np.clip(img, 0, 1)
    plt.imshow(img)
    plt.axis('off')
    plt.show()

# ثامناً: دالة مرنة لعرض خريطة الأهمية
def visualize_attr(attr):
    attr = attr.cpu().detach().numpy()

    # معالجة الأبعاد المختلفة
    if attr.ndim == 4:
        attr = attr.squeeze(0)
    if attr.ndim == 3 and attr.shape[0] == 1:
        attr = attr.squeeze(0)
    elif attr.ndim == 3 and attr.shape[0] == 3:
        attr = attr.transpose(1, 2, 0)
        attr = np.sum(np.abs(attr), axis=2)
    elif attr.ndim == 2:
        pass
    else:
        raise ValueError(f"شكل غير متوقع للبيانات: {attr.shape}")

    # تطبيع القيم
    attr = (attr - attr.min()) / (attr.max() - attr.min() + 1e-8)

    # العرض
    plt.imshow(attr, cmap='hot')
    plt.colorbar()
    plt.axis('off')
    plt.title('خريطة الأهمية')
    plt.show()

# تاسعاً: اختيار عينة عشوائية من البيانات
idx = random.randint(0, len(test_dataset)-1)
image, label = test_dataset[idx]
image = image.to(device)
label = torch.tensor(label).to(device)

# عاشراً: حساب خريطة الأهمية
ig = IntegratedGradients(model)
attributions, delta = ig.attribute(
    inputs=image.unsqueeze(0),
    target=label,
    return_convergence_delta=True
)

# الحادي عشر: عرض النتائج
print(f"Convergence Delta: {delta.item():.4f}")
show_image(image)
visualize_attr(attributions)


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

def evaluate_model(model, test_loader):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            _, preds = torch.max(outputs, 1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # حساب المقاييس
    print(classification_report(all_labels, all_preds, target_names=class_names, digits=4))

    # طباعة confusion matrix
    cm = confusion_matrix(all_labels, all_preds)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=class_names, yticklabels=class_names)
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.title("Confusion Matrix")
    plt.show()

# استدعاء الدالة على test_loader
evaluate_model(mobilenet, test_loader)

In [ ]:
# https://drive.google.com/file/d/1LD6v33J7z5Vt4CnVOAs4kWICHIt5JLyg/view?usp=sharing